### --- Day 17: Pyroclastic Flow ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/17

In [2]:
#!import ../Utils.ipynb

In [3]:
var inputLines = LoadPuzzleInput(2022, 17);
WriteLines(inputLines, maxCols: 50);

Loading puzzle file: Day17.txt
Total lines: 1
Max line length: 10091

>>>><<<<>>>><>>>><<<>><<<>>><<><<>>><<>>><<<>>>><>


In [4]:
string[] testInputLines = [
    ">>><<><>><<<>><>>><<<>>><<<><<<>><>><<>>"
];

In [5]:
string[] shapes = [
    "####",
    "",
    ".#.",
    "###",
    ".#.",
    "",
    "..#",
    "..#",
    "###",
    "",
    "#",
    "#",
    "#",
    "#",
    "",
    "##",
    "##",
];

In [6]:
HashSet<Point> ParseShape(string[] shapeLines)
{
    CharGrid g = new(shapeLines);
    return new(g.Enumerate().Where(gg => gg.ch is '#').Select(gg => gg.point));
}

var shapePoints = shapes.SeparateBy(s => s is "").Select(ParseShape).ToList();
shapePoints.Display();

index value 0 [ (0, 0), (1, 0), (2, 0), (3, 0) ] Count 4 Capacity 7 Comparer System.Collections.Generic.GenericEqualityComparer`1[Submission#12+Point] (values) index value 0 (0, 0) X 0 Y 0 1 (1, 0) X 1 Y 0 2 (2, 0) X 2 Y 0 3 (3, 0) X 3 Y 0 1 [ (1, 0), (0, 1), (1, 1), (2, 1), (1, 2) ] Count 5 Capacity 7 Comparer System.Collections.Generic.GenericEqualityComparer`1[Submission#12+Point] (values) index value 0 (1, 0) X 1 Y 0 1 (0, 1) X 0 Y 1 2 (1, 1) X 1 Y 1 3 (2, 1) X 2 Y 1 4 (1, 2) X 1 Y 2 2 [ (2, 0), (2, 1), (0, 2), (1, 2), (2, 2) ] Count 5 Capacity 7 Comparer System.Collections.Generic.GenericEqualityComparer`1[Submission#12+Point] (values) index value 0 (2, 0) X 2 Y 0 1 (2, 1) X 2 Y 1 2 (0, 2) X 0 Y 2 3 (1, 2) X 1 Y 2 4 (2, 2) X 2 Y 2 3 [ (0, 0), (0, 1), (0, 2), (0, 3) ] Count 4 Capacity 7 Comparer System.Collections.Generic.GenericEqualityComparer`1[Submission#12+Point] (values) index value 0 (0, 0) X 0 Y 0 1 (0, 1) X 0 Y 1 2 (0, 2) X 0 Y 2 3 (0, 3) X 0 Y 3 4 [ (0, 0), (1, 0), (0, 1), (1, 1) ] Count 4 Capacity 7 Comparer System.Collections.Generic.GenericEqualityComparer`1[Submission#12+Point] (values) index value 0 (0, 0) X 0 Y 0 1 (1, 0) X 1 Y 0 2 (0, 1) X 0 Y 1 3 (1, 1) X 1 Y 1

For this one, I'm thinking of leveraging the `HashSet<>.Overlaps` method - we'll model the tower as one `HashSet`, and each falling rock as another. Each manipulation of the falling rocks (ie sideways / down), we can manipulate the point point values of the falling rock's `HashSet`. If this causes an overlap, we stop this rock and move to the next one.

In [7]:
// Helper method to apply movements to a rock

static void Apply(this HashSet<Point> rock, Point direction)
{
    var newPoints = rock.Select(r => r + direction).ToArray();
    
    rock.Clear();
    foreach (var n in newPoints)
    {
        rock.Add(n);
    }
}

In [8]:
class TowerSim(IList<HashSet<Point>> rockShapes, string[] inputLines)
{
    public HashSet<Point> Tower { get; } = new();
    
    public int TowerHeight => Tower.Select(p => -p.Y + 1).Append(0).Max();

    public string Jetstream { get; } = inputLines[0];

    public void DoSomeFalling(int rockCount)
    {
        foreach (var rock in MakeRockSequence(rockCount))
        {
            HashSet<Point> nextRock = new(rock);
            var startPoint = FindStartPoint(nextRock);
            nextRock.Apply(startPoint);

            foreach (var move in NextShapeMove())
            {
                HashSet<Point> maybeShape = new(nextRock);
                maybeShape.Apply(move);

                if (IsOOB(maybeShape) || Tower.Overlaps(maybeShape))
                {
                    if (move == Down) { break; }
                    continue;
                }

                nextRock = maybeShape;
            }
            Tower.UnionWith(nextRock);
        }
    }

    public int RockSequenceTotal { get; private set; } = 0;
    public int RockSequenceIndex => RockSequenceTotal % rockShapes.Count;
    IEnumerable<HashSet<Point>> MakeRockSequence(int rockCount)
        => Enumerable.Range(0, rockCount)
            .Select(_ => rockShapes[RockSequenceTotal++ % rockShapes.Count]);

    Point FindStartPoint(HashSet<Point> nextShape)
    {
        // The tall, vertical chamber is exactly seven units wide. Each rock
        // appears so that its left edge is two units away from the left wall
        // and its bottom edge is three units above the highest rock in the room
        // (or the floor, if there isn't one).

        var height = nextShape.Select(p => p.Y).Distinct().Count();

        Point result = (0,0);
        result += Right * 2;
        result += Up * (height + TowerHeight + 2);

        return result;
    }

    public int MoveSequenceTotal { get; private set; } = 0;
    public int MoveSequenceIndex => MoveSequenceTotal % Jetstream.Length;
    IEnumerable<Point> NextShapeMove()
    {
        foreach (var _ in Enumerable.Range(0, int.MaxValue))
        {
            var ch = Jetstream[MoveSequenceIndex];
            var dir = ch is '<' ? Left : Right;
            MoveSequenceTotal++;
            yield return dir;
            yield return Down;
        }
        throw new Exception($"This shouldn't happen!");
    }

    bool IsOOB(HashSet<Point> shape) => shape.Any(p => p switch 
    {
        (< 0 or > 6, _) => true,
        (_, > 0) => true,
        _ => false
    });
}

In [9]:
int DoSomeFalling2(string[] inputLines, int rockCount)
{
    TowerSim towerSim = new(shapePoints, inputLines);
    towerSim.DoSomeFalling(rockCount);
    return towerSim.TowerHeight;
}
var testResult = DoSomeFalling2(inputLines, 2022);
Console.WriteLine(testResult);

3224


In [10]:
// To prove to the elephants your simulation is accurate, they want to know how
// tall the tower will get after 2022 rocks have stopped (but before the 2023rd
// rock begins falling). In this example, the tower of rocks will be 3068 units
// tall.

// var testAnswer = DoSomeFalling(testInputLines, 2022).Last().towerHeight;
var testAnswer = DoSomeFalling2(testInputLines, 2022);
Console.WriteLine(testAnswer);

3068


In [11]:
// How many units tall will the tower of rocks be after 2022 rocks have stopped
// falling?

// var part1Answer = DoSomeFalling(inputLines, 2022).Last().towerHeight;
var part1Answer = DoSomeFalling2(inputLines, 2022);
Console.WriteLine(part1Answer);

3224


In [12]:
// 3224 is correct!
Ensure(3224, part1Answer);

### --- Part Two ---

Puzzle description redacted as-per Advent of Code guidelines

You may find the puzzle description at: https://adventofcode.com/2022/day/17

Geez, 1,000,000,000,000 rocks huh? I suspect this is going to require some kind of cycle. We know the rock sequence is repeating, so perhaps after we have done the entire cycle we know the rock pattern will repeat itself. Although that might depend on the drop patterns. Perhaps we can find the point at which the rocks and drop patterns both cycle back to the start, and we'll check that.

But what would a cycle look like in this case? 

* We are starting at the same rock (so the rock order will be the same)
* We are starting at the same jetstream position (so the dropping movements will be the same)
* The columns of each tower are the same height (so we know they will land in the same configuration)

For the columns configuration part, let's define a "signature", which is the number of whitespaces before the tops of each column of the tower. Eg, if the tower had a top like so...

```
#.#.#..
######.
#######
```

...the signature would be `0,1,0,1,0,2`. This should uniquely describe each possible towertop configuration. 

Let's check every batch of rocks (5), and see if we can find a repeating jetstream position + signature - this should be our repetition.

In [14]:
// A checkpoint to inspect after each batch of rocks
using Checkpoint = (int towerHeight, int rockSequenceIndex, int rockSequenceTotal, int moveSequenceIndex, string signature);

// Our signature function as explained above
string GetSignature(HashSet<Point> tower)
{
    if (tower.Count is 0) { return "empty"; }

    // Tower ascends in -Y, so highest point is lowest Y
    var yMax = tower.Min(p => p.Y);
    var xMax = 6;

    var mins = Enumerable.Range(0, xMax + 1).Select(x => new Point(x, 0));

    var q = from p in tower.Concat(mins)
            group p by p.X into pGroup
            orderby pGroup.Key
            select Math.Abs(yMax - pGroup.Select(pg => pg.Y).Min());

    return string.Join(",", q.Select(qq => qq.ToString()));
}

Console.WriteLine(GetSignature([]));

HashSet<Point> testPoints = [
    (0, -1),
    (1, -2)
];
Console.WriteLine(GetSignature(testPoints));

empty
1,0,2,2,2,2,2


In [15]:
IEnumerable<Checkpoint> GetCheckpoints(string[] inputLines, int rockCount)
{
    TowerSim towerSim = new(shapePoints, inputLines);

    var remain = rockCount;
    while (remain > 0)
    {
        var nextChunk = Math.Min(remain, shapePoints.Count);
        towerSim.DoSomeFalling(nextChunk);

        Checkpoint cp = (
            towerSim.TowerHeight,
            towerSim.RockSequenceIndex,
            towerSim.RockSequenceTotal,
            towerSim.MoveSequenceIndex,
            GetSignature(towerSim.Tower)
        );

        remain -= nextChunk;
        yield return cp;
    }
}

foreach (var checkpoint in GetCheckpoints(testInputLines, 100))
{
    Console.WriteLine(checkpoint);
}

(9, 0, 5, 24, 5,5,3,5,0,0,8)
(17, 0, 10, 12, 3,3,4,4,0,2,16)
(25, 0, 15, 2, 7,11,5,4,4,0,0)
(36, 0, 20, 28, 18,4,4,6,0,10,11)
(43, 0, 25, 15, 25,0,0,6,3,3,1)
(51, 0, 30, 5, 0,4,2,4,5,5,5)
(60, 0, 35, 34, 9,4,4,6,0,8,14)
(66, 0, 40, 21, 15,0,0,0,4,3,4)
(72, 0, 45, 10, 4,3,2,2,0,4,2)
(78, 0, 50, 2, 10,5,0,0,0,2,3)
(89, 0, 55, 28, 21,4,4,6,0,10,14)
(96, 0, 60, 15, 28,0,0,6,3,3,1)
(104, 0, 65, 5, 0,4,2,4,5,5,5)
(113, 0, 70, 34, 9,4,4,6,0,8,14)
(119, 0, 75, 21, 15,0,0,0,4,3,4)
(125, 0, 80, 10, 4,3,2,2,0,4,2)
(131, 0, 85, 2, 10,5,0,0,0,2,3)
(142, 0, 90, 28, 21,4,4,6,0,10,14)
(149, 0, 95, 15, 28,0,0,6,3,3,1)
(157, 0, 100, 5, 0,4,2,4,5,5,5)


Looks like there is some repetition in the test input - eg the signature `0,4,2,4,5,5,5`  always repeats back at pos `5` in the move sequence. Let's continue!

In [16]:
void FindRepetition(string[] inputLines, int rockCount)
{
    Dictionary<(string signature, int jetstreamPos), int> repeatCount = new();

    foreach (var checkpoint in GetCheckpoints(inputLines, rockCount))
    {
        var sig = checkpoint.signature;
        var moveIdx = checkpoint.moveSequenceIndex;
        var sigMove = (sig, moveIdx);

        repeatCount[sigMove] = repeatCount.TryGetValue(sigMove, out var count) switch {
            true => count + 1,
            _ => 1
        };
    }

    repeatCount.OrderByDescending(r => r.Value).Take(5).ToList().Display();
}

Console.WriteLine("Repetitions in test input:");
FindRepetition(testInputLines, 2022);
Console.WriteLine("Repetitions in puzzle input");
FindRepetition(inputLines, 10_000);

Repetitions in test input:


index value 0 [(0,4,2,4,5,5,5, 5), 57] Key (0,4,2,4,5,5,5, 5) Item1 0,4,2,4,5,5,5 Item2 5 Value 57 1 [(9,4,4,6,0,8,14, 34), 57] Key (9,4,4,6,0,8,14, 34) Item1 9,4,4,6,0,8,14 Item2 34 Value 57 2 [(15,0,0,0,4,3,4, 21), 57] Key (15,0,0,0,4,3,4, 21) Item1 15,0,0,0,4,3,4 Item2 21 Value 57 3 [(4,3,2,2,0,4,2, 10), 57] Key (4,3,2,2,0,4,2, 10) Item1 4,3,2,2,0,4,2 Item2 10 Value 57 4 [(10,5,0,0,0,2,3, 2), 57] Key (10,5,0,0,0,2,3, 2) Item1 10,5,0,0,0,2,3 Item2 2 Value 57

Repetitions in puzzle input


index value 0 [(6,6,0,4,4,9,30, 819), 6] Key (6,6,0,4,4,9,30, 819) Item1 6,6,0,4,4,9,30 Item2 819 Value 6 1 [(2,2,0,0,0,1,36, 847), 6] Key (2,2,0,0,0,1,36, 847) Item1 2,2,0,0,0,1,36 Item2 847 Value 6 2 [(9,9,4,4,0,8,31, 1131), 6] Key (9,9,4,4,0,8,31, 1131) Item1 9,9,4,4,0,8,31 Item2 1131 Value 6 3 [(22,22,8,8,0,0,44, 1151), 6] Key (22,22,8,8,0,0,44, 1151) Item1 22,22,8,8,0,0,44 Item2 1151 Value 6 4 [(12,10,0,4,2,3,3, 1193), 6] Key (12,10,0,4,2,3,3, 1193) Item1 12,10,0,4,2,3,3 Item2 1193 Value 6

Ok I'm fairly confident we're going to find repetitive cycles now. Let's try to specifically find a cycle.

In [17]:
void FindRepetitionCycle(string[] inputLines, int rockCount)
{
    Dictionary<(string signature, int moveSequenceIndex), Checkpoint> found = new();

    (string signature, int moveSequenceIndex) firstRepeat = default;
    foreach (var checkpoint in GetCheckpoints(inputLines, rockCount))
    {
        var (_, _, _, moveSequenceIndex, signature) = checkpoint;

        if (found.ContainsKey((signature, moveSequenceIndex)))
        {
            var sig = (signature, moveSequenceIndex);
            if (sig == firstRepeat || firstRepeat == default)
            {
                var previous = found[sig];
                if (firstRepeat == default)
                {
                    Console.WriteLine("Original was...");
                    Console.WriteLine(previous);
                }
                
                Console.WriteLine($"We found a repetition!");
                Console.WriteLine(checkpoint);

                var heightDiff = checkpoint.towerHeight - previous.towerHeight;
                var shapeDiff = checkpoint.rockSequenceTotal - previous.rockSequenceTotal;
                Console.WriteLine($"Height diff: {heightDiff}. Rocks diff: {shapeDiff}");

                firstRepeat = sig;
            } 
        }
        found[(signature, moveSequenceIndex)] = checkpoint;
    }
}

Console.WriteLine("Finding repetitive cycle in test input:");
FindRepetitionCycle(testInputLines, 200);
Console.WriteLine();
Console.WriteLine("Finding repetitive cycle in puzzle input");
FindRepetitionCycle(inputLines, 6000); 

Finding repetitive cycle in test input:
Original was...
(51, 0, 30, 5, 0,4,2,4,5,5,5)
We found a repetition!
(104, 0, 65, 5, 0,4,2,4,5,5,5)
Height diff: 53. Rocks diff: 35
We found a repetition!
(157, 0, 100, 5, 0,4,2,4,5,5,5)
Height diff: 53. Rocks diff: 35
We found a repetition!
(210, 0, 135, 5, 0,4,2,4,5,5,5)
Height diff: 53. Rocks diff: 35
We found a repetition!
(263, 0, 170, 5, 0,4,2,4,5,5,5)
Height diff: 53. Rocks diff: 35

Finding repetitive cycle in puzzle input
Original was...
(188, 0, 130, 819, 6,6,0,4,4,9,30)
We found a repetition!
(2989, 0, 1880, 819, 6,6,0,4,4,9,30)
Height diff: 2801. Rocks diff: 1750
We found a repetition!
(5774, 0, 3625, 819, 6,6,0,4,4,9,30)
Height diff: 2785. Rocks diff: 1745
We found a repetition!
(8559, 0, 5370, 819, 6,6,0,4,4,9,30)
Height diff: 2785. Rocks diff: 1745


Ok, we found the cycle we are looking for! In the test input this worked beautifully, but there's a slight quirk in the puzzle input. It seems the cycle interval isn't quite right the first time, i.e., `1750` vs `1745` thereafter. After some thinking I suspect this is due to my definition of the signature: something like a "C" shape would have the same signature as a straight column, so some falling rocks might drift and settle in the gap. The first instance of the cycle is found after only 130 blocks, building up from the flat base of the tower. Perhaps this underlying rock formation is slightly different from the subsequent cycles which are building up from the jagged top of the tower.

I could try and code-out this quirk but I'll bury my head in the sand for now and just process a bunch of rocks before searching for cycles :) But at least let's confirm our suspiction and compare the towers.

In [18]:
void CheckTowerShapes()
{
    TowerSim sim = new(shapePoints, inputLines);
    sim.DoSomeFalling(130);
    Console.WriteLine($"After falling, we have: {GetSignature(sim.Tower)}");
    sim.Tower.Render();

    var firstHeight = sim.TowerHeight;
    sim.DoSomeFalling(1750);
    Console.WriteLine($"After some more falling, we have: {GetSignature(sim.Tower)}");
    sim.Tower.Where(p => p.Y <= -sim.TowerHeight + firstHeight).Render();

    sim.DoSomeFalling(1745);
    Console.WriteLine($"After falling a third time, we have {GetSignature(sim.Tower)}");
    sim.Tower.Where(p => p.Y <= -sim.TowerHeight + firstHeight).Render();
}

// Skipping this verbose output, but it confirms the towers differ between cycles 1-2, but are identical between 2-3
// CheckTowerShapes();

Now that we know there are cycles, we can put it all together and calculate the height mostly as a function of the cycle length.

In [19]:
const long repetitions = 1_000_000_000_000;

long DoLargeFall(string[] inputLines, long rockCount)
{
    Dictionary<(string signature, int jetstreamIndex), (int towerHeight, int shapeTotal)> repeatLookup = new();

    long heightIncrement = 0;
    long rockIncrement = 0;

    TowerSim towerSim = new(shapePoints, inputLines);

    // Head-burying-in-sand moment here: see comments above
    towerSim.DoSomeFalling(5000);

    while (heightIncrement is 0)
    {
        towerSim.DoSomeFalling(shapePoints.Count);
        var sigIndex = (GetSignature(towerSim.Tower), towerSim.MoveSequenceIndex);
        if (repeatLookup.TryGetValue(sigIndex, out var previous))
        {
            // we found the repeat!
            var (prevTowerheight, prevShapeTotal) = previous;

            heightIncrement = towerSim.TowerHeight - prevTowerheight;
            rockIncrement = towerSim.RockSequenceTotal - prevShapeTotal;
            break;
        }

        repeatLookup[sigIndex] = (towerSim.TowerHeight, towerSim.RockSequenceTotal);
    }

    long remainingDrops = rockCount - towerSim.RockSequenceTotal;
    long remainingLoops = remainingDrops / rockIncrement;
    int remainingManualSteps = (int)(remainingDrops % rockIncrement);
    towerSim.DoSomeFalling(remainingManualSteps);
    long result = towerSim.TowerHeight + (remainingLoops * heightIncrement);
    
    return result;
}

In [20]:
// In the example above, the tower would be 1514285714288 units tall!

var part2TestAnswer = DoLargeFall(testInputLines, repetitions);
Console.WriteLine(part2TestAnswer);
Ensure(1514285714288L, part2TestAnswer);

1514285714288


In [21]:
// How tall will the tower be after 1000000000000 rocks have stopped?

var part2Answer = DoLargeFall(inputLines, repetitions);
Console.WriteLine(part2Answer);

1595988538691


In [22]:
// 1595988538691 is correct!
Ensure(1595988538691L, part2Answer);